# Lê Hoàng Thiên Hùng
# 2174802010513

### Câu 1 (2 điểm): Cho một ảnh bất kỳ (tên ảnh do sinh viên tự đặt, ví dụ: `my_image.jpg`) và thực hiện các yêu cầu sau:

* Viết chương trình sử dụng Bilateral filter để làm mịn ảnh. (0.5 điểm)  
* Viết chương trình sử dụng Canny Edge Detection để xác định biên của hình ảnh. (0.5 điểm)  
* Đổi màu ảnh bằng cách hoán đổi kênh màu theo thứ tự (ví dụ: BGR → BRG) và lưu thành tên dạng `[ten_anh]_swapped.jpg`. (0.5 điểm)  
* Chuyển ảnh sang không gian màu YCrCb và tách riêng 3 kênh Y, Cr, Cb, lưu thành ảnh grayscale tương ứng (`[ten_anh]_Y.jpg`, `[ten_anh]_Cr.jpg`, `[ten_anh]_Cb.jpg`). (0.5 điểm)


In [3]:
import cv2
import os


image_name = 'bird.png'

img = cv2.imread(image_name)
if img is None:
    raise FileNotFoundError(f"Không tìm thấy ảnh: {image_name}")

base_name = os.path.splitext(image_name)[0]

bilateral = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)
cv2.imwrite(f'{base_name}_bilateral.jpg', bilateral)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, threshold1=100, threshold2=200)
cv2.imwrite(f'{base_name}_edges.jpg', edges)

b, g, r = cv2.split(img)
swapped = cv2.merge([b, r, g])  # BRG
cv2.imwrite(f'{base_name}_swapped.jpg', swapped)

ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
Y, Cr, Cb = cv2.split(ycrcb)
cv2.imwrite(f'{base_name}_Y.jpg', Y)
cv2.imwrite(f'{base_name}_Cr.jpg', Cr)
cv2.imwrite(f'{base_name}_Cb.jpg', Cb)

print("Xử lý ảnh hoàn tất.")


Xử lý ảnh hoàn tất.


### Câu 2 (4 điểm) Viết một chương trình Python sử dụng OpenCV để tạo menu tương tác cho phép người dùng chọn các kỹ thuật biến đổi hình học và xử lý ảnh nâng cao từ một danh sách, áp dụng đồng thời cho nhiều ảnh.

### Yêu cầu:

1. Menu gồm:  
* Phóng to ảnh (Zoom bằng resize) (0.5 điểm)  
* Xoay ảnh (góc ngẫu nhiên từ 0–360 độ) (0.5 điểm)  
* Lật ảnh ngang (0.5 điểm)  
* Lật ảnh dọc (0.5 điểm)  
* Cắt ảnh (crop ngẫu nhiên vùng giữa ảnh) (0.5 điểm)  
* Thêm viền (padding màu ngẫu nhiên) (0.5 điểm)

2. Chương trình xử lý đồng thời 3 ảnh bất kỳ do sinh viên tự chọn (có thể chọn bằng đường dẫn file hoặc nhập tên ảnh tùy ý). (0.5 điểm)

3. Phím tương ứng để kích hoạt các phương pháp xử lý:  
* Z: Zoom  
* T: Rotate  
* H: Horizontal Flip  
* V: Vertical Flip  
* C: Crop  
* P: Padding (0.5 điểm)

4. Lưu file kết quả với định dạng: `result_[phương pháp]_[tên ảnh gốc].jpg`  
   Ví dụ: `result_crop_cat.jpg`, `result_rotate_image1.jpg` (0.5 điểm)


In [ ]:
import cv2
import numpy as np
import os
import random

# ---- HÀM XỬ LÝ ẢNH ----
def zoom_image(img, scale=1.5):
    height, width = img.shape[:2]
    return cv2.resize(img, (int(width * scale), int(height * scale)))

def rotate_image(img):
    angle = random.randint(0, 360)
    height, width = img.shape[:2]
    center = (width // 2, height // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1)
    return cv2.warpAffine(img, M, (width, height))

def flip_horizontal(img):
    return cv2.flip(img, 1)

def flip_vertical(img):
    return cv2.flip(img, 0)

def crop_image(img):
    h, w = img.shape[:2]
    new_h, new_w = int(h * 0.5), int(w * 0.5)
    start_y = h // 4
    start_x = w // 4
    return img[start_y:start_y + new_h, start_x:start_x + new_w]

def add_padding(img):
    top, bottom = 30, 30
    left, right = 30, 30
    color = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
    return cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

# ---- NHẬP ẢNH ----
image_paths = []
print("Nhập đường dẫn hoặc tên 3 ảnh:")
for i in range(3):
    path = input(f"Ảnh {i+1}: ")
    if not os.path.exists(path):
        raise FileNotFoundError(f"Không tìm thấy ảnh: {path}")
    image_paths.append(path)

# ---- ĐỌC ẢNH ----
images = [cv2.imread(p) for p in image_paths]
names = [os.path.splitext(os.path.basename(p))[0] for p in image_paths]

# ---- MENU TƯƠNG TÁC ----
print("\nNhấn phím để chọn phương pháp xử lý:")
print("Z - Zoom\nT - Rotate\nH - Flip ngang\nV - Flip dọc\nC - Crop\nP - Padding\nQ - Thoát")

while True:
    key = input("Chọn phương pháp (Z/T/H/V/C/P/Q): ").upper()
    if key == 'Q':
        print("Thoát chương trình.")
        break

    method = None
    processed_images = []

    if key == 'Z':
        method = 'zoom'
        processed_images = [zoom_image(img) for img in images]
    elif key == 'T':
        method = 'rotate'
        processed_images = [rotate_image(img) for img in images]
    elif key == 'H':
        method = 'flipH'
        processed_images = [flip_horizontal(img) for img in images]
    elif key == 'V':
        method = 'flipV'
        processed_images = [flip_vertical(img) for img in images]
    elif key == 'C':
        method = 'crop'
        processed_images = [crop_image(img) for img in images]
    elif key == 'P':
        method = 'padding'
        processed_images = [add_padding(img) for img in images]
    else:
        print("Phím không hợp lệ! Vui lòng thử lại.")
        continue

    # ---- LƯU KẾT QUẢ ----
    for i in range(3):
        out_name = f"result_{method}_{names[i]}.jpg"
        cv2.imwrite(out_name, processed_images[i])
        print(f"Đã lưu: {out_name}")


Nhập đường dẫn hoặc tên 3 ảnh:



Nhấn phím để chọn phương pháp xử lý:
Z - Zoom
T - Rotate
H - Flip ngang
V - Flip dọc
C - Crop
P - Padding
Q - Thoát
Phím không hợp lệ! Vui lòng thử lại.
Phím không hợp lệ! Vui lòng thử lại.
Phím không hợp lệ! Vui lòng thử lại.
Phím không hợp lệ! Vui lòng thử lại.
Đã lưu: result_zoom_bird.jpg
Đã lưu: result_zoom_flower.jpg
Đã lưu: result_zoom_orange.jpg
Đã lưu: result_rotate_bird.jpg
Đã lưu: result_rotate_flower.jpg
Đã lưu: result_rotate_orange.jpg
Đã lưu: result_flipH_bird.jpg
Đã lưu: result_flipH_flower.jpg
Đã lưu: result_flipH_orange.jpg
Phím không hợp lệ! Vui lòng thử lại.
Phím không hợp lệ! Vui lòng thử lại.
Phím không hợp lệ! Vui lòng thử lại.
Phím không hợp lệ! Vui lòng thử lại.
Phím không hợp lệ! Vui lòng thử lại.


### Câu 3 (4 điểm) Viết một chương trình Python để xử lý 3 ảnh bất kỳ do sinh viên tự chọn.

* Thêm viền đen 20 pixel cho ảnh đầu tiên. (0.5 điểm)  
* Xoay ảnh thứ hai 45 độ và phóng to 1.5 lần. (0.5 điểm)  
* Tăng kích thước ảnh thứ ba lên 4 lần, sau đó áp dụng Bilateral Filter với tham số tùy chọn. (1.5 điểm)  
* Thay đổi độ sáng và độ tương phản ảnh thứ ba theo công thức:

$$
I_{out}(x, y) = \alpha \cdot I_{in}(x, y) + \beta
$$

Trong đó:  

$$
\alpha \in [0.6, 2.0], \quad \beta \in [-60, 60]
$$

Giá trị đầu ra cần được giới hạn trong khoảng [0, 255] bằng công thức:

$$
I_{out}(x, y) = \text{clip}(I_{out}(x, y), 0, 255)
$$


# Chúc các bạn thi may mắn và đạt điểm 10.

In [ ]:
import cv2
import numpy as np
import os

def adjust_brightness_contrast(img, alpha=1.0, beta=0):
    result = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    return result

image_paths = []
print("Nhập đường dẫn hoặc tên 3 ảnh:")
for i in range(3):
    path = input(f"Ảnh {i+1}: ")
    if not os.path.exists(path):
        raise FileNotFoundError(f"Không tìm thấy ảnh: {path}")
    image_paths.append(path)

img1 = cv2.imread(image_paths[0])
img2 = cv2.imread(image_paths[1])
img3 = cv2.imread(image_paths[2])

name1 = os.path.splitext(os.path.basename(image_paths[0]))[0]
name2 = os.path.splitext(os.path.basename(image_paths[1]))[0]
name3 = os.path.splitext(os.path.basename(image_paths[2]))[0]

bordered_img1 = cv2.copyMakeBorder(img1, 20, 20, 20, 20, cv2.BORDER_CONSTANT, value=(0, 0, 0))
cv2.imwrite(f"result_border_{name1}.jpg", bordered_img1)

h2, w2 = img2.shape[:2]
center = (w2 // 2, h2 // 2)
M = cv2.getRotationMatrix2D(center, 45, 1.5)
rotated_scaled_img2 = cv2.warpAffine(img2, M, (w2, h2))
cv2.imwrite(f"result_rotate_scale_{name2}.jpg", rotated_scaled_img2)

resized_img3 = cv2.resize(img3, None, fx=4, fy=4, interpolation=cv2.INTER_LINEAR)

bilateral_img3 = cv2.bilateralFilter(resized_img3, d=9, sigmaColor=75, sigmaSpace=75)

alpha = 1.8   # Trong khoảng [0.6, 2.0]
beta = 40     # Trong khoảng [-60, 60]
bright_contrast_img3 = adjust_brightness_contrast(bilateral_img3, alpha=alpha, beta=beta)

cv2.imwrite(f"result_bilateral_{name3}.jpg", bilateral_img3)
cv2.imwrite(f"result_bright_contrast_{name3}.jpg", bright_contrast_img3)

print("\n✅ Xử lý hoàn tất! Các ảnh đã được lưu.")


Nhập đường dẫn hoặc tên 3 ảnh:

✅ Xử lý hoàn tất! Các ảnh đã được lưu.
